In [1]:
import numpy as np
import pandas as pd
from scipy import signal
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
class RELM:
    def __init__(self, n_hidden=100, activation='tanh', C=1.0, random_state=None):
        self.n_hidden = int(n_hidden)
        self.activation = activation
        self.C = float(C)
        self.random_state = random_state
        self.is_fitted = False

    def _init_weights(self, n_features):
        rng = np.random.default_rng(self.random_state)
        self.W = rng.uniform(-1, 1, size=(self.n_hidden, n_features))
        self.b = rng.uniform(-1, 1, size=(self.n_hidden,))

    def _activation(self, X):
        if self.activation == 'sigmoid':
            X = np.clip(X, -500, 500)
            return 1.0 / (1.0 + np.exp(-X))
        if self.activation == 'tanh':
            return np.tanh(X)
        if self.activation == 'relu':
            return np.maximum(0.0, X)
        raise ValueError("Unknown activation")

    def fit(self, X, y):
        X = np.asarray(X); y = np.asarray(y)
        if y.ndim == 1: y = y.reshape(-1, 1)
        N, d = X.shape
        self._init_weights(d)
        H = self._activation(X @ self.W.T + self.b)
        # Ridge-regularized least squares (two cases for stability)
        if N >= self.n_hidden:
            A = (np.eye(self.n_hidden) / self.C) + (H.T @ H)
            B = H.T @ y
            self.beta = np.linalg.solve(A, B)
        else:
            A = (np.eye(N) / self.C) + (H @ H.T)
            B = y
            self.beta = H.T @ np.linalg.solve(A, B)
        self.is_fitted = True
        return self

    def predict(self, X):
        H = self._activation(np.asarray(X) @ self.W.T + self.b)
        Y = H @ self.beta
        return Y.ravel() if Y.shape[1] == 1 else Y

In [4]:
class GWO:
    def __init__(self, obj_func, lb, ub, dim, n_agents=12, n_iter=25, seed=42):
        self.obj = obj_func
        self.lb = np.array(lb, dtype=float)
        self.ub = np.array(ub, dtype=float)
        self.dim = dim
        self.n_agents = n_agents
        self.n_iter = n_iter
        self.rng = np.random.default_rng(seed)

    def optimize(self):
        wolves = self.rng.uniform(self.lb, self.ub, size=(self.n_agents, self.dim))
        fitness = np.array([self.obj(w) for w in wolves])
        idx = np.argsort(fitness)
        alpha, beta, delta = wolves[idx[0]].copy(), wolves[idx[1]].copy(), wolves[idx[2]].copy()
        f_alpha, f_beta, f_delta = float(fitness[idx[0]]), float(fitness[idx[1]]), float(fitness[idx[2]])
        for t in range(self.n_iter):
            a = 2 - 2 * (t / (self.n_iter - 1 + 1e-9))
            for i in range(self.n_agents):
                X = wolves[i].copy()
                for j in range(self.dim):
                    r1, r2 = self.rng.random(), self.rng.random()
                    A1 = 2 * a * r1 - a; C1 = 2 * r2
                    D_alpha = abs(C1 * alpha[j] - X[j]); X1 = alpha[j] - A1 * D_alpha

                    r1, r2 = self.rng.random(), self.rng.random()
                    A2 = 2 * a * r1 - a; C2 = 2 * r2
                    D_beta = abs(C2 * beta[j] - X[j]); X2 = beta[j] - A2 * D_beta

                    r1, r2 = self.rng.random(), self.rng.random()
                    A3 = 2 * a * r1 - a; C3 = 2 * r2
                    D_delta = abs(C3 * delta[j] - X[j]); X3 = delta[j] - A3 * D_delta

                    X[j] = (X1 + X2 + X3) / 3.0
                wolves[i] = np.clip(X, self.lb, self.ub)
            fitness = np.array([self.obj(w) for w in wolves])
            idx = np.argsort(fitness)
            if fitness[idx[0]] < f_alpha:
                alpha, f_alpha = wolves[idx[0]].copy(), float(fitness[idx[0]])
            if fitness[idx[1]] < f_beta:
                beta, f_beta = wolves[idx[1]].copy(), float(fitness[idx[1]])
            if fitness[idx[2]] < f_delta:
                delta, f_delta = wolves[idx[2]].copy(), float(fitness[idx[2]])
        return alpha, f_alpha

In [5]:
# =====================================
# db4 analysis/synthesis filter taps
# (orthonormal Daubechies-4; length 8)
# =====================================
def db4_filters():
    # Decomposition low-pass (h0) and high-pass (h1)
    h0 = np.array([
        -0.010597401784997278,
         0.032883011666982945,
         0.030841381835560763,
        -0.18703481171888114,
        -0.027983769416859854,
         0.6308807679298589,
         0.7148465705525415,
         0.23037781330885523
    ], dtype=float)
    # High-pass from low-pass with quadrature mirror property
    h1 = np.array([(-1)**k * h0[::-1][k] for k in range(len(h0))], dtype=float)

    # Reconstruction filters (g0, g1) for orthonormal wavelets
    g0 = h0[::-1]  # synthesis low-pass
    g1 = -h1[::-1] # synthesis high-pass
    return h0, h1, g0, g1

In [6]:
def conv_reflect_same(x, h):
    """
    1D convolution with 'symmetric' / reflect padding so that output length == len(x).
    """
    m = len(h)
    pad = m - 1
    xpad = np.pad(x, (pad, pad), mode='reflect')
    y = signal.convolve(xpad, h, mode='valid')  # valid yields len(xpad) - m + 1 = len(x)+pad
    # Correct length to exactly len(x)
    if len(y) > len(x):
        # Center crop
        start = (len(y) - len(x)) // 2
        y = y[start:start+len(x)]
    elif len(y) < len(x):
        y = np.pad(y, (0, len(x)-len(y)), mode='constant')
    return y

def downsample2(x, phase=0):
    return x[phase::2]

def upsample2(x):
    y = np.zeros(2 * len(x), dtype=float)
    y[::2] = x
    return y

In [7]:
def wpd_db4_decompose(signal_in, level=3):
    """
    Build full WPD binary tree to `level` using db4 filters.
    Returns:
      leaves: dict path -> coeff array  (paths are strings of 'a'/'d', length==level)
      all_nodes: dict path -> coeff array  (includes intermediate nodes)
    """
    h0, h1, _, _ = db4_filters()
    x0 = np.asarray(signal_in, dtype=float)
    nodes = {'': x0}  # root path ''
    # For each level, split every current node into 'a' and 'd'
    for L in range(1, level+1):
        new_nodes = {}
        for path, x in nodes.items():
            # Analysis filtering + downsampling
            approx = downsample2(conv_reflect_same(x, h0), phase=0)
            detail = downsample2(conv_reflect_same(x, h1), phase=0)
            new_nodes[path + 'a'] = approx
            new_nodes[path + 'd'] = detail
        nodes = new_nodes
    # At the end, nodes are just the leaves at depth `level`
    leaves = nodes
    # Build dict of all nodes for potential debugging (optional)
    return leaves

In [8]:
def reconstruct_leaf_db4(leaf_coeffs, path):
    """
    Reconstruct time-domain contribution of a single leaf given its coefficients and path.
    Path is a string like 'aad' meaning at each synthesis step use:
      - g0 if char=='a' (low branch), g1 if 'd' (high branch)
    """
    _, _, g0, g1 = db4_filters()
    y = np.asarray(leaf_coeffs, dtype=float)
    # Walk from leaf back to root (reverse path)
    for ch in reversed(path):
        y_up = upsample2(y)
        if ch == 'a':
            y = conv_reflect_same(y_up, g0)
        else:
            y = conv_reflect_same(y_up, g1)
    return y  # length should be close to original (boundary effects handled by reflect)

def wpd_db4_reconstruct_components(signal_in, level=3):
    """
    Decompose and then reconstruct each leaf component to original length.
    Returns:
      comp_list: list of arrays (each approx original length)
      labels: list of leaf paths in order
    """
    leaves = wpd_db4_decompose(signal_in, level=level)
    labels = sorted(leaves.keys())  # deterministic order (lexicographic)
    comps = []
    N = len(signal_in)
    for p in labels:
        c = leaves[p]
        rec = reconstruct_leaf_db4(c, p)
        # Adjust length with center crop/pad to exactly N
        if len(rec) > N:
            start = (len(rec) - N) // 2
            rec = rec[start:start+N]
        elif len(rec) < N:
            rec = np.pad(rec, (0, N-len(rec)), mode='constant')
        comps.append(rec)
    return comps, labels

In [9]:
def create_multivariate_lagged_dataset(df, target_col, feature_cols, lag=12):
    data = df[feature_cols].values
    target_idx = feature_cols.index(target_col)
    X, y = [], []
    for i in range(lag, len(df)):
        X.append(data[i-lag:i].flatten())
        y.append(data[i, target_idx])
    return np.array(X), np.array(y)

def safe_mape(y_true, y_pred, min_denom=1.0):
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    mask = np.abs(y_true) >= min_denom
    if np.sum(mask) == 0:
        return np.nan
    return float(np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100)

def sde(y_true, y_pred):
    return float(np.std(np.asarray(y_true) - np.asarray(y_pred)))

In [10]:
def decode_relm_params(position, Hmin=20, Hmax=500, Cmin_log=-4, Cmax_log=4):
    h_raw, c_log, a_raw = position
    n_hidden = int(np.round(Hmin + np.clip(h_raw, 0, 1) * (Hmax - Hmin)))
    n_hidden = int(np.clip(n_hidden, Hmin, Hmax))
    C = 10.0 ** float(np.clip(c_log, Cmin_log, Cmax_log))
    act_idx = int(np.round(np.clip(a_raw, 0, 2)))
    activation = ['tanh', 'sigmoid', 'relu'][act_idx]
    return n_hidden, C, activation

def make_relm_objective(X_train, y_train, X_val, y_val, random_state=42,
                        Hmin=20, Hmax=500, Cmin_log=-4, Cmax_log=4):
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_val_s = scaler.transform(X_val)
    def objective(position):
        n_hidden, C, activation = decode_relm_params(position, Hmin=Hmin, Hmax=Hmax, Cmin_log=Cmin_log, Cmax_log=Cmax_log)
        try:
            mdl = RELM(n_hidden=n_hidden, activation=activation, C=C, random_state=random_state)
            mdl.fit(X_train_s, y_train)
            y_p = mdl.predict(X_val_s)
            return float(np.sqrt(mean_squared_error(y_val, y_p)))
        except Exception:
            return 1e6
    return objective

In [11]:
def wpd_gwo_relm_pipeline_scipy(
    df,
    target_column,
    feature_columns,
    lag_steps=12,
    level=3,
    gwo_agents=12,
    gwo_iters=25,
    random_state=42,
    Hmin=20, Hmax=400,
    Cmin_log=-4, Cmax_log=4,
    max_step_eval=7
):
    """
    Returns:
      - per_component_info: tuned params per WPD leaf (path label)
      - one_step_metrics: dict
      - multistep_df: DataFrame with Step, MAE, RMSE, MAPE, SDE
      - components: list of time-domain leaf components
      - labels: list of leaf paths (e.g., 'aad', 'ada', ...)
      - y_true_test, y_pred_reconstructed
    """
    y = df[target_column].values.astype(float)
    # 1) WPD(db4) components
    components, labels = wpd_db4_reconstruct_components(y, level=level)
    n_comp = len(components)
    if n_comp == 0:
        raise RuntimeError("No WPD components produced.")

    # 2) lagged split indices (shared across components)
    N = len(y)
    n_samples = N - lag_steps
    if n_samples <= 0:
        raise ValueError("lag_steps too large for series length.")
    train_end = int(0.7 * n_samples)
    val_end = int(0.85 * n_samples)

    per_component_info = []
    test_comp_preds = []
    test_comp_lengths = []

    # 3) Per-leaf model
    for ci, comp in enumerate(components):
        df_c = df.copy()
        df_c[target_column] = comp  # target is this leaf component

        Xc, yc = create_multivariate_lagged_dataset(df_c, target_column, feature_columns, lag=lag_steps)
        X_train, y_train = Xc[:train_end], yc[:train_end]
        X_val, y_val     = Xc[train_end:val_end], yc[train_end:val_end]
        X_test, y_test   = Xc[val_end:], yc[val_end:]

        if len(X_train) < 5 or len(X_val) < 1 or len(X_test) < 1:
            per_component_info.append({"component": ci+1, "label": labels[ci], "skipped": True})
            test_comp_preds.append(np.zeros_like(X_test[:,0]) if X_test.size else np.array([]))
            test_comp_lengths.append(len(X_test))
            continue

        # GWO tuning
        obj = make_relm_objective(X_train, y_train, X_val, y_val, random_state=random_state,
                                  Hmin=Hmin, Hmax=Hmax, Cmin_log=Cmin_log, Cmax_log=Cmax_log)
        lb = np.array([0.0, Cmin_log, 0.0], dtype=float)
        ub = np.array([1.0, Cmax_log, 2.0], dtype=float)
        gwo = GWO(obj, lb, ub, dim=3, n_agents=gwo_agents, n_iter=gwo_iters, seed=random_state + ci)
        best_pos, best_fit = gwo.optimize()
        n_hidden, C, activation = decode_relm_params(best_pos, Hmin=Hmin, Hmax=Hmax, Cmin_log=Cmin_log, Cmax_log=Cmax_log)

        # Final train on train+val
        scaler = StandardScaler()
        X_trval = np.vstack([X_train, X_val])
        y_trval = np.concatenate([y_train, y_val])
        X_trval_s = scaler.fit_transform(X_trval)
        X_test_s = scaler.transform(X_test)

        mdl = RELM(n_hidden=n_hidden, activation=activation, C=C, random_state=random_state)
        mdl.fit(X_trval_s, y_trval)
        y_pred_test_comp = mdl.predict(X_test_s)

        per_component_info.append({
            "component": ci+1,
            "label": labels[ci],
            "best_val_rmse": float(best_fit),
            "n_hidden": int(n_hidden),
            "C": float(C),
            "activation": activation,
            "test_len": int(len(y_test))
        })
        test_comp_preds.append(y_pred_test_comp)
        test_comp_lengths.append(len(y_test))

    # 4) Reconstruct final test forecast by summation
    # Find common n_test (first non-zero length)
    n_test = next((ln for ln in test_comp_lengths if ln), None)
    if n_test is None:
        raise RuntimeError("No valid test length across components.")
    stacked = []
    for arr in test_comp_preds:
        a = np.asarray(arr)
        if len(a) == n_test:
            stacked.append(a)
        elif len(a) == 0:
            stacked.append(np.zeros(n_test))
        elif len(a) < n_test:
            stacked.append(np.concatenate([a, np.zeros(n_test - len(a))]))
        else:
            stacked.append(a[:n_test])
    stacked = np.array(stacked)
    y_pred_reconstructed = np.sum(stacked, axis=0)
    y_true_test = df[target_column].values[lag_steps + val_end : lag_steps + val_end + n_test]

    # 5) One-step metrics
    one_mae  = mean_absolute_error(y_true_test, y_pred_reconstructed)
    one_rmse = np.sqrt(mean_squared_error(y_true_test, y_pred_reconstructed))
    one_mape = safe_mape(y_true_test, y_pred_reconstructed)
    one_sdev = sde(y_true_test, y_pred_reconstructed)
    one_step_metrics = {"MAE": float(one_mae), "RMSE": float(one_rmse),
                        "MAPE (%)": float(one_mape) if not np.isnan(one_mape) else np.nan,
                        "SDE": float(one_sdev)}

    # 6) Multi-step (direct) metrics (1..H) using TRAIN only with tuned params
    multistep_rows = []
    for step in range(1, max_step_eval + 1):
        comp_step_preds = []
        valid = True
        for info in per_component_info:
            if info.get("skipped", False):
                comp_step_preds.append(np.zeros(max(0, n_test - step)))
                continue
            ci = info["component"] - 1
            comp_series = components[ci]
            df_c = df.copy(); df_c[target_column] = comp_series
            Xc, yc = create_multivariate_lagged_dataset(df_c, target_column, feature_columns, lag=lag_steps)
            X_train, y_train = Xc[:train_end], yc[:train_end]
            X_test_all, y_test_all = Xc[val_end:], yc[val_end:]
            if X_test_all.shape[0] <= step:
                valid = False; break
            X_test_step = X_test_all[:-step]

            n_hidden = info["n_hidden"]; C = info["C"]; activation = info["activation"]
            scaler_tr = StandardScaler()
            X_train_s = scaler_tr.fit_transform(X_train)
            X_test_step_s = scaler_tr.transform(X_test_step)
            mdl = RELM(n_hidden=n_hidden, activation=activation, C=C, random_state=random_state)
            mdl.fit(X_train_s, y_train)
            y_pred_step_comp = mdl.predict(X_test_step_s)
            comp_step_preds.append(y_pred_step_comp)

        if not valid:
            break

        comp_step_preds = np.array(comp_step_preds)
        y_pred_step_recon = np.sum(comp_step_preds, axis=0)
        y_true_step = df[target_column].values[lag_steps + val_end + step :
                                               lag_steps + val_end + step + y_pred_step_recon.shape[0]]
        m = min(len(y_true_step), len(y_pred_step_recon))
        if m == 0:
            break
        y_true_step = y_true_step[:m]; y_pred_step_recon = y_pred_step_recon[:m]
        multistep_rows.append({
            "Step": step,
            "MAE": float(mean_absolute_error(y_true_step, y_pred_step_recon)),
            "RMSE": float(np.sqrt(mean_squared_error(y_true_step, y_pred_step_recon))),
            "MAPE (%)": float(safe_mape(y_true_step, y_pred_step_recon)),
            "SDE": float(sde(y_true_step, y_pred_step_recon))
        })

    multistep_df = pd.DataFrame(multistep_rows)

    return {
        "per_component_info": per_component_info,
        "one_step_metrics": one_step_metrics,
        "multistep_df": multistep_df,
        "components": components,
        "labels": labels,
        "y_true_test": y_true_test,
        "y_pred_reconstructed": y_pred_reconstructed
    }

In [ ]:
feature_columns = ['AirTemp','Azimuth','CloudOpacity','DewpointTemp','Dhi','Dni','Ebh',
                   'WindDirection10m','Ghi','RelativeHumidity','SurfacePressure','WindSpeed10m']
df = pd.read_csv('/Users/hrishityelchuri/Documents/windPred/raw/8.52 hrishit data.csv')

df['PeriodEnd'] = pd.to_datetime(df['PeriodEnd'])
df['PeriodStart'] = pd.to_datetime(df['PeriodStart'])
df = df.sort_values('PeriodEnd')

In [13]:
res = wpd_gwo_relm_pipeline_scipy(
    df,
    target_column='WindSpeed10m',
    feature_columns=feature_columns,
    lag_steps=12,
    level=3,              # depth of packet tree; try 2..5
    gwo_agents=12,
    gwo_iters=25,
    random_state=42,
    Hmin=20, Hmax=400,
    Cmin_log=-4, Cmax_log=4,
    max_step_eval=7
)
print("Per-leaf tuned params:")
print(pd.DataFrame(res['per_component_info']))
print("\nOne-step metrics:", res['one_step_metrics'])
print("\nMulti-step metrics:")
print(res['multistep_df'].to_string(index=False))

Per-leaf tuned params:
   component label  best_val_rmse  n_hidden             C activation  test_len
0          1   aaa       0.419117       395   6406.834868       relu     20612
1          2   aad       0.103625       386   2414.919644       relu     20612
2          3   ada       0.024640       398      0.004908       relu     20612
3          4   add       0.065155       394     11.393172       relu     20612
4          5   daa       0.003505       398      0.028129       relu     20612
5          6   dad       0.008660       396   2010.817517       relu     20612
6          7   dda       0.016320       398    621.080933       relu     20612
7          8   ddd       0.015858       399  10000.000000       relu     20612

One-step metrics: {'MAE': 0.9793318821636319, 'RMSE': 1.208473003386571, 'MAPE (%)': 37.61190542872876, 'SDE': 1.2029115094272225}

Multi-step metrics:
 Step      MAE     RMSE  MAPE (%)      SDE
    1 1.206712 1.459650 47.083456 1.396769
    2 1.286172 1.557432 49.